**Increasing the revenues of restaurants by analysing the population in a specific region and their food habits.**

Introduction

To increase the revenues of restaurants by analysing the food habits of the customer with their orders in restaurant and by identifying huge populations new food combinations will be introduced and attract more customers in a specific region.



Data

I will be using Foursquare to utilise location data and explore London to identify the London boroughs that requires to identify the restaurants in a specific region.


Areas will be identified that:

Have large Populations
Have more restaurants 
Have more children.


In [11]:
#London boroughs loaded
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests

url='https://en.wikipedia.org/wiki/List_of_London_boroughs'

B=pd.read_html(url, header=0)[0]

B.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [6]:
#Cleaning the data up
#Dropping irrelevant columns
B = B.drop(['Inner','Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
#Cleaning up borough names
B['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
B['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
B['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
B.head(10)

,Borough,Area (sq mi),Population (2013 est)[1],Co-ordinates
0,Barking and Dagenham,13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E
1,Barnet,33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E
5,Camden,8.40,229719,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W
6,Croydon,33.41,372752,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W
7,Ealing,21.44,342494,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W
8,Enfield,31.74,320524,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W
9,Greenwich,18.28,264008,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E


In [8]:
#Renaming the column titles
B = B.rename(columns = {"Area (sq mi)": "Area", 
                        "Population (2013 est)[1]":"Population",})
#Fixing coords
geolocator = Nominatim(user_agent="London_explorer")
B['Co-ordinates']= B['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
B[['Latitude', 'Longitude']] = B['Co-ordinates'].apply(pd.Series)
Longitude_list = B['Longitude'].tolist()
Latitude_list = B['Latitude'].tolist()
print ("Old latitude list: ", Latitude_list)
print ("Old Longitude list: ", Longitude_list)

replace_longitudes = {-106.6621329:0.0799, -2.8417544: 0.1837}
replace_latitudes = {50.7164496:51.6636, 51.0358628: 51.5499}

longtitudes_new = [replace_longitudes.get(n9,n9) for n9 in Longitude_list]
latitudes_new = [replace_latitudes.get(n9,n9) for n9 in Latitude_list]

B = B.drop(['Co-ordinates', 'Longitude'], axis=1)

B['Longitude'] = longtitudes_new
B['Latitude'] = latitudes_new
B.head(10)

Old latitude list:  [51.5541171, 51.65309, 39.9692378, 32.9373463, 51.4028046, 39.9448402, 51.3713049, 51.5126553, 51.6520851, 51.4820845, 51.5432402, 51.4920377, 51.587929849999995, 51.5967688, 51.0043613, 51.542519299999995, 51.4686132, 51.5384287, 51.4875417, 51.4096275, 51.5013012, 51.4624325, 51.410802849999996, 51.52999955, 51.5763203, 51.44046095, 51.5029222, 30.5672947, 51.1288633, 42.3756401, 51.4570271, 39.8366528]
Old Longitude list:  [0.15050434261994267, -0.2002261, -82.936864, -87.1647184, 0.0148142, -75.1198911, -0.101957, -0.3051952, -0.0810175, -0.0045417, -0.0493621, -0.2236401, -0.10541010599099046, -0.33727515543507014, -2.337474942629507, -0.44833493117949663, -0.3613471, -0.0999051, -0.1682205, -0.3062621, -0.117287, -0.0101331, -0.18809850595572736, 0.02931796029382208, 0.0454097, -0.3055192359117199, -0.103458, -100.6432359, 1.2986686, -71.2358004, -0.1932607, -105.0372046]


,Borough,Area,Population,Latitude,Longitude
0,Barking and Dagenham,13.93,194352,51.554117,0.150504
1,Barnet,33.49,369088,51.653090,-0.200226
2,Bexley,23.38,236687,39.969238,-82.936864
3,Brent,16.70,317264,32.937346,-87.164718
4,Bromley,57.97,317899,51.402805,0.014814
5,Camden,8.40,229719,39.944840,-75.119891
6,Croydon,33.41,372752,51.371305,-0.101957
7,Ealing,21.44,342494,51.512655,-0.305195
8,Enfield,31.74,320524,51.652085,-0.081018
9,Greenwich,18.28,264008,51.482084,-0.004542


In [9]:
#All boroughs on a Map
from geopy.geocoders import Nominatim
address = 'London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [10]:
!pip install folium
import folium

Final = folium.Map(location=[London_latitude, London_longitude], zoom_start=10)


for lat, lng, label in zip(B['Latitude'], B['Longitude'], 
                            B['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='Yellow',
        fill=True,
        fill_color='#Black',
        fill_opacity=0.5).add_to(Final)
Final

Results

The restaurant spots are identified in the folium map by the areas where the population is in huge numbers and thereby revenues can be increased with setup of new restaurants and if there are restaurants already present in that area, new food items are introduced in the menu.  